In [3]:
import pandas as pd
import requests
import numpy as np

In [11]:
base_url = 'https://api-web.nhle.com/v1'

In [27]:
def GetTeamSeasonStats(team_abbrev):
    url = f'{base_url}/club-stats/{team_abbrev}/now'
    req = requests.get(url)
    return req.json()


GetTeamSeasonStats('TOR').keys()

dict_keys(['season', 'gameType', 'skaters', 'goalies'])